In [59]:
'''
Tools for Manipulating and Converting 3D Rotations
By Omid Alemi
Created: June 12, 2017
Adapted from that matlab file...
'''

import math
import numpy as np

def deg2rad(x):
    return x/180*math.pi


def rad2deg(x):
    return x/math.pi*180

class Rotation():
    def __init__(self,rot, param_type, rotation_order, **params):
        self.rotmat = []
        self.rotation_order = rotation_order
        if param_type == 'euler':
            self._from_euler(rot[0],rot[1],rot[2], params)
        elif param_type == 'expmap':
            self._from_expmap(rot[0], rot[1], rot[2], params)

    def _from_euler(self, alpha, beta, gamma, params):
        '''Expecting degress'''

        if params['from_deg']==True:
            alpha = deg2rad(alpha)
            beta = deg2rad(beta)
            gamma = deg2rad(gamma)
        
        ca = math.cos(alpha)
        cb = math.cos(beta)
        cg = math.cos(gamma)
        sa = math.sin(alpha)
        sb = math.sin(beta)
        sg = math.sin(gamma)        

        Rx = np.asarray([[1, 0, 0], 
              [0, ca, sa], 
              [0, -sa, ca]
              ])

        Ry = np.asarray([[cb, 0, -sb], 
              [0, 1, 0],
              [sb, 0, cb]])

        Rz = np.asarray([[cg, sg, 0],
              [-sg, cg, 0],
              [0, 0, 1]])

        self.rotmat = np.eye(3)

        ############################ inner product rotation matrix in order defined at BVH file #########################
        for axis in self.rotation_order :
            if axis == 'X' :
                self.rotmat = np.matmul(Rx, self.rotmat)
            elif axis == 'Y':
                self.rotmat = np.matmul(Ry, self.rotmat)
            else :
                self.rotmat = np.matmul(Rz, self.rotmat)
        ################################################################################################################
   
    def _from_expmap(self, alpha, beta, gamma, params):
        if (alpha == 0 and beta == 0 and gamma == 0):
            self.rotmat = np.eye(3)
            return

        #TODO: Check exp map params

        theta = np.linalg.norm([alpha, beta, gamma])

        expmap = [alpha, beta, gamma] / theta

        x = expmap[0]
        y = expmap[1]
        z = expmap[2]

        s = math.sin(theta/2)
        c = math.cos(theta/2)

        self.rotmat = np.asarray([
            [2*(x**2-1)*s**2+1,  2*x*y*s**2-2*z*c*s,  2*x*z*s**2+2*y*c*s],
            [2*x*y*s**2+2*z*c*s,  2*(y**2-1)*s**2+1,  2*y*z*s**2-2*x*c*s],
            [2*x*z*s**2-2*y*c*s, 2*y*z*s**2+2*x*c*s , 2*(z**2-1)*s**2+1]
        ])
        


    def get_euler_axis(self):
        R = self.rotmat
        theta = math.acos((self.rotmat.trace() - 1) / 2)
        axis = np.asarray([R[2,1] - R[1,2], R[0,2] - R[2,0], R[1,0] - R[0,1]])
        axis = axis/(2*math.sin(theta))
        return theta, axis

    def to_expmap(self):
        theta, axis = self.get_euler_axis()
        rot_arr = theta * axis
        if np.isnan(rot_arr).any():
            rot_arr = [0, 0, 0]
        return rot_arr
    
    def to_euler(self, use_deg=False):
        eulers = np.zeros((2, 3))

        if np.absolute(np.absolute(self.rotmat[2, 0]) - 1) < 1e-12:
            #GIMBAL LOCK!
            print('Gimbal')
            if np.absolute(self.rotmat[2, 0]) - 1 < 1e-12:
                eulers[:,0] = math.atan2(-self.rotmat[0,1], -self.rotmat[0,2])
                eulers[:,1] = -math.pi/2
            else:
                eulers[:,0] = math.atan2(self.rotmat[0,1], -elf.rotmat[0,2])
                eulers[:,1] = math.pi/2
            
            return eulers

        theta = - math.asin(self.rotmat[2,0])
        theta2 = math.pi - theta

        # psi1, psi2
        eulers[0,0] = math.atan2(self.rotmat[2,1]/math.cos(theta), self.rotmat[2,2]/math.cos(theta))
        eulers[1,0] = math.atan2(self.rotmat[2,1]/math.cos(theta2), self.rotmat[2,2]/math.cos(theta2))

        # theta1, theta2
        eulers[0,1] = theta
        eulers[1,1] = theta2

        # phi1, phi2
        eulers[0,2] = math.atan2(self.rotmat[1,0]/math.cos(theta), self.rotmat[0,0]/math.cos(theta))
        eulers[1,2] = math.atan2(self.rotmat[1,0]/math.cos(theta2), self.rotmat[0,0]/math.cos(theta2))

        if use_deg:
            eulers = rad2deg(eulers)

        return eulers
    
    def to_quat(self):
        #TODO
        pass
    
    def __str__(self):
        return "Rotation Matrix: \n " + self.rotmat.__str__()
    


In [3]:
import sys
sys.path.append('..')

In [4]:
from kpt.model.bvh_model import BVHModel
from kpt.model.robot_model import RobotModel
from kpt.vis.scatter_animation import scatter_animation, scatter_scene
import numpy as np
from pymo.preprocessing import *
from pymo.parsers import BVHParser


In [6]:
path = '../sample_data/bvh/01_01.bvh'
parsed = BVHParser().parse(path)
mp = MocapParameterizer('position')

In [7]:
mp

MocapParameterizer(param_type='position')

In [8]:
parsed

In [10]:
euler_df = parsed.values

In [11]:
pos_df = pd.DataFrame(index=euler_df.index)

In [66]:
rot_cols = [c for c in euler_df.columns if ('rotation' in c)]
rot_cols

['Hips_Zrotation',
 'Hips_Yrotation',
 'Hips_Xrotation',
 'LHipJoint_Zrotation',
 'LHipJoint_Yrotation',
 'LHipJoint_Xrotation',
 'LeftUpLeg_Zrotation',
 'LeftUpLeg_Yrotation',
 'LeftUpLeg_Xrotation',
 'LeftLeg_Zrotation',
 'LeftLeg_Yrotation',
 'LeftLeg_Xrotation',
 'LeftFoot_Zrotation',
 'LeftFoot_Yrotation',
 'LeftFoot_Xrotation',
 'LeftToeBase_Zrotation',
 'LeftToeBase_Yrotation',
 'LeftToeBase_Xrotation',
 'RHipJoint_Zrotation',
 'RHipJoint_Yrotation',
 'RHipJoint_Xrotation',
 'RightUpLeg_Zrotation',
 'RightUpLeg_Yrotation',
 'RightUpLeg_Xrotation',
 'RightLeg_Zrotation',
 'RightLeg_Yrotation',
 'RightLeg_Xrotation',
 'RightFoot_Zrotation',
 'RightFoot_Yrotation',
 'RightFoot_Xrotation',
 'RightToeBase_Zrotation',
 'RightToeBase_Yrotation',
 'RightToeBase_Xrotation',
 'LowerBack_Zrotation',
 'LowerBack_Yrotation',
 'LowerBack_Xrotation',
 'Spine_Zrotation',
 'Spine_Yrotation',
 'Spine_Xrotation',
 'Spine1_Zrotation',
 'Spine1_Yrotation',
 'Spine1_Xrotation',
 'Neck_Zrotation',
 'N

In [67]:
pos_cols = [c for c in euler_df.columns if ('position' in c)]
pos_cols

['Hips_Xposition', 'Hips_Yposition', 'Hips_Zposition']

In [68]:
joints = (joint for joint in parsed.skeleton)
joints

<generator object <genexpr> at 0x7fdfc958cc10>

In [69]:
len(list(joints))

38

In [70]:
tree_data = {}

In [71]:
joint = 'LeftUpLeg'
parent = parsed.skeleton[joint]['parent']
rc = euler_df[[c for c in rot_cols if joint in c]]
pc = euler_df[[c for c in pos_cols if joint in c]]

In [72]:
rc

,LeftUpLeg_Zrotation,LeftUpLeg_Yrotation,LeftUpLeg_Xrotation
0 days 00:00:00,-21.0000,0.0000,0.0000
0 days 00:00:00.008333300,-13.8102,2.5002,3.3502
0 days 00:00:00.016666600,-13.9682,2.5229,3.4357
0 days 00:00:00.024999900,-13.8218,2.5041,3.5075
0 days 00:00:00.033333200,-14.1244,2.3274,3.6587
...,...,...,...
0 days 00:00:22.891575100,-10.2469,3.3432,-7.5606
0 days 00:00:22.899908400,-9.9686,3.6640,-8.2394
0 days 00:00:22.908241700,-9.7540,3.9225,-8.9724
0 days 00:00:22.916575,-8.8645,4.1468,-9.9555


In [73]:
pc

""
0 days 00:00:00
0 days 00:00:00.008333300
0 days 00:00:00.016666600
0 days 00:00:00.024999900
0 days 00:00:00.033333200
...
0 days 00:00:22.891575100
0 days 00:00:22.899908400
0 days 00:00:22.908241700
0 days 00:00:22.916575


In [76]:
rc.shape

(2752, 3)

In [77]:
euler_values = [[f[1]['%s_Xrotation'%joint], 
                                     f[1]['%s_Yrotation'%joint], 
                                     f[1]['%s_Zrotation'%joint]] for f in rc.iterrows()]

In [78]:
euler_values

[[0.0, 0.0, -21.0],
 [3.3502, 2.5002, -13.8102],
 [3.4357, 2.5229, -13.9682],
 [3.5075, 2.5041, -13.8218],
 [3.6587, 2.3274, -14.1244],
 [3.7263, 2.2258, -14.1889],
 [3.6317, 2.2992, -14.0111],
 [3.4675, 2.2152, -14.0037],
 [3.2614, 2.0739, -13.6701],
 [2.9934, 2.0239, -12.933],
 [2.7195, 1.958, -12.6724],
 [2.5459, 1.9037, -12.8399],
 [2.5077, 1.7424, -12.9495],
 [2.4636, 1.4917, -12.9865],
 [2.3346, 1.4626, -12.7883],
 [2.1426, 1.6119, -12.3962],
 [2.057, 1.653, -12.4355],
 [2.0902, 1.5035, -12.8431],
 [2.0012, 1.3264, -12.9144],
 [1.9324, 1.2681, -12.7266],
 [1.933, 1.0977, -13.0274],
 [1.8176, 0.8055, -13.7071],
 [1.6503, 0.752, -13.8522],
 [1.4783, 0.8091, -13.4461],
 [1.3234, 0.6032, -12.9667],
 [1.1932, 0.3022, -12.6334],
 [0.9987, 0.1558, -12.517],
 [0.6914, 0.062, -12.5759],
 [0.3829, -0.0383, -12.9136],
 [0.1607, -0.2049, -13.2756],
 [-0.0768, -0.3286, -13.2227],
 [-0.5351, -0.1991, -13.1248],
 [-0.2756, -0.7741, -13.0004],
 [-1.2336, -0.5159, -13.3535],
 [-1.7285, -0.7146, -

In [79]:
rc.columns

Index(['LeftUpLeg_Zrotation', 'LeftUpLeg_Yrotation', 'LeftUpLeg_Xrotation'], dtype='object')

In [80]:
rotation_order = rc.columns[0][rc.columns[0].find('rotation') - 1] + rc.columns[1][rc.columns[1].find('rotation') - 1] + rc.columns[2][rc.columns[2].find('rotation') - 1] #rotation_order is string : 'XYZ' or'ZYX' or ...
rotation_order

'ZYX'

In [81]:
pc.shape

(2752, 0)

In [82]:
if pc.shape[1] < 3:
    pos_values = [[0,0,0] for f in pc.iterrows()]

In [83]:
rotmats = np.asarray([Rotation([f[0], f[1], f[2]], 'euler', rotation_order, from_deg=True).rotmat for f in euler_values])

In [86]:
rotmats

array([[[ 0.93358043, -0.35836795,  0.        ],
        [ 0.35836795,  0.93358043,  0.        ],
        [ 0.        ,  0.        ,  1.        ]],

       [[ 0.97016739, -0.23847911, -0.04362287],
        [ 0.24077396,  0.96882368,  0.05838308],
        [ 0.02833973, -0.06714461,  0.99734069]],

       [[ 0.96948922, -0.24114936, -0.04401868],
        [ 0.24350945,  0.96804891,  0.05987026],
        [ 0.02817456, -0.06876254,  0.99723513]],

       ...,

       [[ 0.98323559, -0.16902144, -0.06840708],
        [ 0.15683078,  0.97529217, -0.15559333],
        [ 0.09301549,  0.14225657,  0.98544974]],

       [[ 0.98546885, -0.1536948 , -0.07231214],
        [ 0.13942563,  0.97510418, -0.17243065],
        [ 0.09701357,  0.15984287,  0.98236379]],

       [[ 0.98814334, -0.13594393, -0.07135817],
        [ 0.12034652,  0.97441323, -0.18983039],
        [ 0.09533864,  0.17899193,  0.97922032]]])

In [33]:
for joint in parsed.traverse():
    parent = parsed.skeleton[joint]['parent']
    rc = euler_df[[c for c in rot_cols if joint in c]]
    pc = euler_df[[c for c in pos_cols if joint in c]]
